# Снятие омонимии, классика

In [9]:
!pip install russian-tagsets
!pip install sklearn-crfsuite

Defaulting to user installation because normal site-packages is not writeable
^C
ERROR: Operation cancelled by user
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import re
from copy import deepcopy
import os

import pandas as pd
import numpy as np

import sklearn_crfsuite

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

from russian_tagsets import converters
from pymorphy2 import MorphAnalyzer
import conllu

from tqdm import tqdm

In [ ]:
p_analyser = MorphAnalyzer()

In [ ]:
t = p_analyser.parse("стекло")[0].tag
t

OpencorporaTag('NOUN,inan,neut sing,nomn')

In [ ]:
to_ud = converters.converter('opencorpora-int', 'ud20')
to_ud(str(t))

'NOUN Animacy=Inan|Case=Nom|Gender=Neut|Number=Sing'

In [ ]:
def __conllu_word_to_dict(tokens, pos) -> dict:
    res = {}
    for i in range(pos-2, pos+3):
        for feat in ['lemma', 'upos', 'feats']:
            if i >= 0 and i < len(tokens):
                res[feat+str(i-pos)] = tokens[i][feat]
            else:
                res[feat+str(i-pos)] = 'none'
    return res


def conllu_word_to_dict(tokens, pos) -> dict:
    res = {}
    out_pos = tokens[pos]['upos']
    for i in range(pos-2, pos+3):
        if i >= 0 and i < len(tokens):
            m_res = p_analyser.parse(tokens[i]['form'])
            m_pos = 0
            while m_pos < len(m_res) and m_pos < 3:
                res['lemma'+str(i-pos)+str(m_pos)] = m_res[m_pos].normal_form
                feats = to_ud(str(m_res[m_pos].tag))
                upos, feats = feats.split(' ')
                res['upos'+str(i-pos)+str(m_pos)] = upos
                res['feats'+str(i-pos)+str(m_pos)] = feats
                m_pos += 1
                
            while m_pos < 3:
                res['lemma'+str(i-pos)+str(m_pos)] = 'none'
                res['upos'+str(i-pos)+str(m_pos)] = 'none'
                res['feats'+str(i-pos)+str(m_pos)] = 'none'
                m_pos += 1
        else:
            for m_pos in range(3):
                res['lemma'+str(i-pos)+str(m_pos)] = 'none'
                res['upos'+str(i-pos)+str(m_pos)] = 'none'
                res['feats'+str(i-pos)+str(m_pos)] = 'none'
        
    return res, out_pos

In [ ]:
ud_path = "/home/edward/projects/Alien_bases/Universal Dependencies/ud-treebanks-v2.8/UD_Russian-SynTagRus/"
in_file = open(ud_path+'ru_syntagrus-ud-train.conllu', 'rt')
train_connlu_iter = conllu.parse_incr(in_file)

In [ ]:
i = 0
for sent in train_connlu_iter:
    print(sent[0], sent, i)
    i += 1
    if i > 5:
        break

Анкета TokenList<Анкета, ., metadata={sent_id: "2003Anketa.xml_1", text: "Анкета."}> 0
Начальник TokenList<Начальник, областного, управления, связи, Семен, Еремеевич, был, человек, простой, ,, приходил, на, работу, всегда, вовремя, ,, здоровался, с, секретаршей, за, руку, и, иногда, даже, писал, в, стенгазету, заметки, под, псевдонимом, ", Муха, ", ., metadata={sent_id: "2003Anketa.xml_2", text: "Начальник областного управления связи Семен Еремеевич был человек простой, приходил на работу всегда вовремя, здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом "Муха"."}> 1
В TokenList<В, приемной, его, с, утра, ожидали, посетители, ,, -, кое-кто, с, важными, делами, ,, а, кое-кто, и, с, такими, ,, которые, легко, можно, было, решить, в, нижестоящих, инстанциях, ,, не, затрудняя, Семена, Еремеевича, ., metadata={sent_id: "2003Anketa.xml_3", text: "В приемной его с утра ожидали посетители, - кое-кто с важными делами, а кое-кто и с такими, которые легко мо

In [ ]:
res_sents = []
res_poses = []

for sent in tqdm(train_connlu_iter):
    tagged_sent = []
    sent_poses = []
    for i in range(len(sent)):
        tags, pos = conllu_word_to_dict(sent, i)
        tagged_sent.append(tags)
        sent_poses.append(pos)
    res_sents.append(tagged_sent)
    res_poses.append(sent_poses)

48808it [10:00, 81.29it/s] 


In [ ]:
tags

{'lemma-20': 'пол',
 'upos-20': 'PROPN',
 'feats-20': 'Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing',
 'lemma-21': 'пол',
 'upos-21': 'NOUN',
 'feats-21': 'Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing',
 'lemma-22': 'пол',
 'upos-22': 'NOUN',
 'feats-22': 'Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing',
 'lemma-10': 'путь',
 'upos-10': 'NOUN',
 'feats-10': 'Animacy=Inan|Case=Loc|Gender=Masc|Number=Sing',
 'lemma-11': 'путь',
 'upos-11': 'NOUN',
 'feats-11': 'Animacy=Inan|Case=Dat|Gender=Masc|Number=Sing',
 'lemma-12': 'путь',
 'upos-12': 'NOUN',
 'feats-12': 'Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing',
 'lemma00': '…',
 'upos00': 'PUNCT',
 'feats00': '_',
 'lemma01': 'none',
 'upos01': 'none',
 'feats01': 'none',
 'lemma02': 'none',
 'upos02': 'none',
 'feats02': 'none',
 'lemma10': 'none',
 'upos10': 'none',
 'feats10': 'none',
 'lemma11': 'none',
 'upos11': 'none',
 'feats11': 'none',
 'lemma12': 'none',
 'upos12': 'none',
 'feats12': 'none',
 'lemma20': 'none',
 'upos20': 'non

In [ ]:
list(zip(tagged_sent, sent_poses))

[({'lemma-20': 'none',
   'upos-20': 'none',
   'feats-20': 'none',
   'lemma-21': 'none',
   'upos-21': 'none',
   'feats-21': 'none',
   'lemma-22': 'none',
   'upos-22': 'none',
   'feats-22': 'none',
   'lemma-10': 'none',
   'upos-10': 'none',
   'feats-10': 'none',
   'lemma-11': 'none',
   'upos-11': 'none',
   'feats-11': 'none',
   'lemma-12': 'none',
   'upos-12': 'none',
   'feats-12': 'none',
   'lemma00': '-',
   'upos00': 'PUNCT',
   'feats00': '_',
   'lemma01': 'none',
   'upos01': 'none',
   'feats01': 'none',
   'lemma02': 'none',
   'upos02': 'none',
   'feats02': 'none',
   'lemma10': 'да',
   'upos10': 'PART',
   'feats10': '_',
   'lemma11': 'да',
   'upos11': 'CCONJ',
   'feats11': '_',
   'lemma12': 'да',
   'upos12': 'INTJ',
   'feats12': '_',
   'lemma20': ',',
   'upos20': 'PUNCT',
   'feats20': '_',
   'lemma21': 'none',
   'upos21': 'none',
   'feats21': 'none',
   'lemma22': 'none',
   'upos22': 'none',
   'feats22': 'none'},
  'PUNCT'),
 ({'lemma-20': 'no

In [ ]:
model = sklearn_crfsuite.CRF(algorithm='lbfgs', c1=0.1, c2=0.1, 
                                   max_iterations=200, all_possible_transitions=True)

In [ ]:
%%time
try:
    model.fit(res_sents, res_poses)
except:
    pass

CPU times: user 9min 52s, sys: 627 ms, total: 9min 52s
Wall time: 16min 44s


In [ ]:
model.predict([tagged_sent])

[['CCONJ',
  'PRON',
  'VERB',
  'PRON',
  'ADV',
  'CCONJ',
  'PUNCT',
  'VERB',
  'VERB',
  'PUNCT',
  'VERB',
  'ADV',
  'NUM',
  'NOUN',
  'PUNCT']]

In [ ]:
sent

TokenList<Но, кто-то, идет, мне, навстречу, и, ,, может, быть, ,, прошел, уже, пол, пути, …, metadata={sent_id: "uppsalaNagibin_4.xml_71", text: "Но кто-то идет мне навстречу и, может быть, прошел уже полпути…"}>

### Pipeline

Для того, чтобы не писать какждый раз руками одну и ту же (а бывает, что и сильно более сложную) последователньость действий токенизация-модель-интерпретация, можно использовать пайплайны.

Они бывают самые разные, полный список можно найти [вот здесь](https://huggingface.co/docs/transformers/main_classes/pipelines).

In [ ]:
from transformers import pipeline
import torch

2024-04-26 13:03:57.195819: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-26 13:03:57.245258: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device = torch.device("cpu")

In [ ]:
oracle = pipeline(
    model='deepset/roberta-base-squad2',
    framework='pt',
    batch_size=10,
    device=device
    )


In [ ]:

oracle(question="Where do I live?", context="My name is Wolfgang and I live in Berlin")

{'score': 0.9190723299980164, 'start': 34, 'end': 40, 'answer': 'Berlin'}

In [ ]:
text = '''Owls are birds from the order Strigiformes[1] (/ˈstrɪdʒəfɔːrmiːz/), which includes over 200 species of mostly solitary and nocturnal birds of prey typified by an upright stance, a large, broad head, binocular vision, binaural hearing, sharp talons, and feathers adapted for silent flight. Exceptions include the diurnal northern hawk-owl and the gregarious burrowing owl.

Owls are divided into two families: the true (or typical) owl family, Strigidae, and the barn-owl family, Tytonidae.[2] Owls hunt mostly small mammals, insects, and other birds, although a few species specialize in hunting fish. They are found in all regions of the Earth except the polar ice caps and some remote islands.

A group of owls is called a "parliament".[3]
Owls possess large, forward-facing eyes and ear-holes, a hawk-like beak, a flat face, and usually a conspicuous circle of feathers, a facial disc, around each eye. The feathers making up this disc can be adjusted to sharply focus sounds from varying distances onto the owls' asymmetrically placed ear cavities. Most birds of prey have eyes on the sides of their heads, but the stereoscopic nature of the owl's forward-facing eyes permits the greater sense of depth perception necessary for low-light hunting. Owls have binocular vision, but they must rotate their entire heads to change the focus of their view because, like most birds, their eyes are fixed in their sockets. Owls are farsighted and cannot clearly see anything nearer than a few centimetres of their eyes. Caught prey can be felt by owls with the use of filoplumes—hairlike feathers on the beak and feet that act as "feelers". Their far vision, particularly in low light, is exceptionally good.

Owls can rotate their heads and necks as much as 270°. Owls have 14 neck vertebrae — humans have only seven — and their vertebral circulatory systems are adapted to allow them to rotate their heads without cutting off blood to the brain. Specifically, the foramina in their vertebrae through which the vertebral arteries pass are about 10 times the diameter of the artery, instead of about the same size as the artery, as is the case in humans; the vertebral arteries enter the cervical vertebrae higher than in other birds, giving the vessels some slack, and the carotid arteries unite in a very large anastomosis or junction, the largest of any bird's, preventing blood supply from being cut off while they rotate their necks. Other anastomoses between the carotid and vertebral arteries support this effect.[4][5]'''

In [ ]:
summarizer_bart = pipeline(
    'summarization',
    framework='pt',
    batch_size=10,
    device=device
    )

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
summarizer_bart(text)

[{'summary_text': ' The true (or typical) owl family, Strigidae, and the barn-owl family, Tytonidae, are divided into two families . Owls hunt mostly small mammals, insects, and other birds, although a few species specialize in hunting fish . They are found in all regions of the Earth except polar ice caps and some remote islands .'}]

In [ ]:
summarizer_t5 = pipeline(
    'summarization',
    model='google-t5/t5-base',
    tokenizer='google-t5/t5-base',
    framework='pt',
    device=device
    )

In [ ]:
summarizer_t5(text)

[{'summary_text': 'owls hunt mostly small mammals, insects, and other birds, although a few species specialize in hunting fish . they have binocular vision, binaural hearing, sharp talons, and feathers adapted for silent flight . birds can rotate their heads and necks as much as 270° .'}]

# Определение тональности, Bert

In [ ]:
model = pipeline(model="seara/rubert-tiny2-russian-sentiment")
model("Привет, ты мне нравишься!")

[{'label': 'positive', 'score': 0.9398769736289978}]

In [ ]:
model = pipeline(model="r1char9/rubert-tiny2-ru-go-emotions")
model("Привет, ты мне нравишься!")

[{'label': 'love', 'score': 0.5816417932510376}]

In [ ]:
model("Как же так получилось, я не нахожу слов и чувств, чтобы описать свои эмоции")

[{'label': 'disapproval', 'score': 0.43194231390953064}]

In [ ]:
model("как же так получилось я в полном раздрае")

[{'label': 'sadness', 'score': 0.2147722691297531}]

# Выделение именованных сущностей, Bert

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "Я живу в Екатеренбурге "

ner_results = nlp(example)
print(ner_results)

[{'entity': 'B-MISC', 'score': 0.6903213, 'index': 7, 'word': 'Е', 'start': 9, 'end': 10}, {'entity': 'I-LOC', 'score': 0.6243275, 'index': 15, 'word': '##б', 'start': 17, 'end': 18}]


In [ ]:
example = "My name is Wolfgang and I live in Berlin"
ner_results = nlp(example)
print(ner_results)

[{'entity': 'B-PER', 'score': 0.9990139, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': 0.999645, 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]


In [ ]:
!pip3 install -U openai-whisper

In [1]:
import whisper
from transformers import pipeline



2024-04-30 16:48:09.091994: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-30 16:48:09.335939: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model = whisper.load_model("base")

/home/edward/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [6]:
result = model.transcribe("stream5.mp3")
print(result["text"])

/home/edward/.local/lib/python3.10/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 7 комфорта, максимум безопасности. Ченган Центр Владимир в автограде представляет красовер нового поколения. Ченган СС-35+. Проверен на дороге. Выбирайте Ченган СС-35+. От 2 миллионов 69 900 рублей. Ченган Центр Владимир в автограде. Подрубнее условия приобретения.


In [ ]:
# import whisper

model = whisper.load_model("base")

# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("stream4.mp3")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# print the recognized text
print(result.text)

Detected language: pl


KeyboardInterrupt: 

In [ ]:
!pip3 install torchaudio omegaconf


^C
ERROR: Operation cancelled by user


In [1]:
import torch
from omegaconf import OmegaConf

In [2]:

torch.hub.download_url_to_file('https://raw.githubusercontent.com/snakers4/silero-models/master/models.yml',
                               'latest_silero_models.yml',
                               progress=False)
models = OmegaConf.load('latest_silero_models.yml')

In [3]:
language = 'ru'
model_id = 'v4_ru'
device = torch.device('cpu')

model, example_text = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                     model='silero_tts',
                                     language=language,
                                     speaker=model_id)
model.to(device)  # gpu or cpu

Using cache found in /home/edward/.cache/torch/hub/snakers4_silero-models_master


In [4]:
model.speakers

['aidar', 'baya', 'kseniya', 'xenia', 'eugene', 'random']

In [5]:
sample_rate = 48000
speaker = 'eugene'
put_accent=True
put_yo=True
# example_text = 'В недрах тундры выдры в г+етрах т+ырят в вёдра ядра к+едров.'
example_text = 'Здравствуйте! Сегодняшнюю лекци будет вест+и нейронная сеть.'

audio = model.apply_tts(text=example_text,
                        speaker=speaker,
                        sample_rate=sample_rate,
                        put_accent=put_accent,
                        put_yo=put_yo)


In [6]:
from pydub import AudioSegment
from pydub.playback import play

In [7]:
audio_segment = AudioSegment(
    audio.detach().numpy().tobytes(), 
    frame_rate=sample_rate, # 44100
    sample_width=4, 
    channels=1
)

play(audio_segment)

ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_route.c:877:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
